In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
import optuna
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegressionCV
import joblib
from boruta import BorutaPy
import xgboost as xgb

/Users/prathmesh/asep_project/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load both datasets
original = pd.read_csv("PCOS_data.csv")
new = pd.read_csv("pcos_dataset.csv")

# Set max columns to show is unlimited
pd.set_option('display.max_columns', None)

In [3]:
def preprocess(df):
    df = df.copy()  # avoid SettingWithCopyWarning

    # 1. Clean column names
    df.columns = df.columns.str.strip() \
                           .str.replace(' ', '_') \
                           .str.replace('(', '') \
                           .str.replace(')', '') \
                           .str.replace('.', '') \
                           .str.replace('-', '_') \
                           .str.replace('/', '_')
    df.rename(columns={'II____beta_HCGmIU_mL': 'II_beta_HCG'}, inplace=True)

    # 2. Drop irrelevant columns
    df.drop(columns=['Sl_No', 'Patient_File_No'], inplace=True, errors='ignore')
    df = df.loc[:, ~df.columns.str.startswith('Unnamed')]

    # 3. Merge Age columns
    if 'Age' not in df.columns and 'Age_yrs' in df.columns:
        df.rename(columns={'Age_yrs': 'Age'}, inplace=True)
    elif 'Age' in df.columns and 'Age_yrs' in df.columns:
        df['Age'] = df['Age'].fillna(df['Age_yrs'])
        df.drop(columns=['Age_yrs'], inplace=True)

    # 4. Merge PCOS diagnosis columns
    if 'PCOS_Diagnosis' in df.columns:
        df.rename(columns={'PCOS_Diagnosis': 'PCOS_Y_N'}, inplace=True)

    # 5. Handle missing values
    if 'Marraige_Status_Yrs' in df.columns:
        df.loc[:, 'Marraige_Status_Yrs'] = df['Marraige_Status_Yrs'].fillna(df['Marraige_Status_Yrs'].median())

    if 'Fast_food_Y_N' not in df.columns and 'Fast_food_YN' in df.columns:
        df.rename(columns={'Fast_food_YN': 'Fast_food_Y_N'}, inplace=True)
    if 'Fast_food_Y_N' in df.columns:
        df.loc[:, 'Fast_food_Y_N'] = df['Fast_food_Y_N'].fillna(df['Fast_food_Y_N'].mode()[0])

    # 6. Convert to numeric and fill missing values
    if 'II_beta_HCG' not in df.columns and 'II_beta_HCGmIU_mL' in df.columns:
        df.rename(columns={'II_beta_HCGmIU_mL': 'II_beta_HCG'}, inplace=True)
    if 'II_beta_HCG' in df.columns:
        df.loc[:, 'II_beta_HCG'] = pd.to_numeric(df['II_beta_HCG'], errors='coerce')
        df['II_beta_HCG'] = df['II_beta_HCG'].astype(float)
        df.loc[:, 'II_beta_HCG'] = df['II_beta_HCG'].fillna(df['II_beta_HCG'].median())

    if 'AMHng_mL' in df.columns:
        df.loc[:, 'AMHng_mL'] = pd.to_numeric(df['AMHng_mL'], errors='coerce')
        df['AMHng_mL'] = df['AMHng_mL'].astype(float)
        df.loc[:, 'AMHng_mL'] = df['AMHng_mL'].fillna(df['AMHng_mL'].median())

    return df

In [4]:
# Apply preprocessing
original_clean = preprocess(original)
new_clean = preprocess(new)

# Ensure consistent columns across both
all_columns = list(set(original_clean.columns).union(set(new_clean.columns)))

# Align both dataframes to same columns, fill missing with NaN
original_aligned = original_clean.reindex(columns=all_columns)
new_aligned = new_clean.reindex(columns=all_columns)

# Concatenate datasets
combined_df = pd.concat([original_aligned, new_aligned], ignore_index=True)

In [5]:
# Separate features and target
# X = combined_df.drop(columns=['PCOS_Y_N'])
X = combined_df.drop(columns=['PCOS_Y_N'])
y = combined_df['PCOS_Y_N']

# Initialize KNNImputer (e.g., with 5 nearest neighbors)
knn_imputer = KNNImputer(n_neighbors=5)

X = pd.DataFrame(knn_imputer.fit_transform(X), columns=X.columns)

# Sanitize feature names
X.columns = [str(col).replace(' ', '_')
                        .replace('"', '')
                        .replace("'", '')
                        .replace('[', '')
                        .replace(']', '')
                        .replace('{', '')
                        .replace('}', '')
                        .replace(':', '')
                        .replace(',', '')
                        for col in X.columns]

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

In [7]:
model = xgb.XGBClassifier()
boruta_selector = BorutaPy(model)
boruta_selector.fit(X.values, y.values)


BorutaPy(estimator=XGBClassifier(base_score=None, booster=None, callbacks=None,
                                 colsample_bylevel=None, colsample_bynode=None,
                                 colsample_bytree=None, device=None,
                                 early_stopping_rounds=None,
                                 enable_categorical=False, eval_metric=None,
                                 feature_types=None, feature_weights=None,
                                 gamma=None, grow_policy=None,
                                 importance_type=None,
                                 interaction_constraints=None,
                                 learning_rate=None, max_bin=None,
                                 max_cat_threshold=None, max_cat_to_onehot=None,
                                 max_delta_step=None, max_depth=None,
                                 max_leaves=None, min_child_weight=None,
                                 missing=nan, monotone_constraints=None,
                                 multi_strategy=None, n_estimators=1000,
                                 n_jobs=None, num_parallel_tree=None, ...),
         random_state=RandomState(MT19937) at 0x11005D440)

In [8]:
# Selected features
selected_features = X.columns[boruta_selector.support_].tolist()
print("Selected Features:", selected_features)

Selected Features: ['Menstrual_Irregularity', 'BMI', 'CycleR_I', 'Skin_darkening_Y_N', 'Follicle_No_R', 'Weight_gainY_N', 'PimplesY_N', 'Cycle_lengthdays', 'hair_growthY_N', 'Follicle_No_L', 'Testosterone_Levelng_dL', 'Antral_Follicle_Count']


In [9]:
X = X[selected_features]
X

,Menstrual_Irregularity,BMI,CycleR_I,Skin_darkening_Y_N,Follicle_No_R,Weight_gainY_N,PimplesY_N,Cycle_lengthdays,hair_growthY_N,Follicle_No_L,Testosterone_Levelng_dL,Antral_Follicle_Count
0,0.656038,-1.339000,-0.959987,-0.982136,-1.121520,-1.086229,-1.401411,0.119674,-0.946588,-1.005664,0.920402,-0.416886
1,-0.744823,-0.159445,-0.959987,-0.982136,-0.495882,-1.086229,-1.401411,0.119674,-0.946588,-1.005664,-0.683750,-0.551277
2,1.122992,-0.075192,-0.959987,-0.982136,2.632309,-1.086229,1.414935,0.119674,-0.946588,2.292051,0.412065,0.154275
3,0.656038,0.851601,-0.959987,-0.982136,-1.434340,-1.086229,-1.401411,0.119674,-0.946588,-1.335436,-0.414870,-0.215300
4,-0.277869,-1.170492,-0.959987,-0.982136,-0.808701,-1.086229,-1.401411,0.119674,-0.946588,-1.005664,-0.029305,-0.248898
...,...,...,...,...,...,...,...,...,...,...,...,...
1536,1.122992,-1.528571,-0.959987,0.200726,-0.370755,-1.086229,-1.401411,0.119674,-0.312374,-0.675893,1.824450,0.927022
1537,1.122992,0.683094,-0.358140,-0.390705,-0.183063,0.455096,-0.274872,-0.074859,-0.312374,0.247468,-1.584755,-1.760795
1538,-1.211777,0.556713,-0.358140,-0.982136,-0.370755,-0.058679,-0.838142,0.508740,-0.312374,-0.148258,-1.386899,1.766965
1539,-1.211777,0.346078,0.845554,0.792156,-0.808701,1.482646,-0.274872,0.119674,-0.312374,-0.346121,1.819377,-1.424818


In [10]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y, shuffle=True, random_state=8)

In [11]:
def objective(trial):
    
    param = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
        "objective": "binary:logistic",
        "eval_metric": "logloss",
        "random_state": 8,
        "use_label_encoder": False,
    }

    train_pool = xgb.DMatrix(x_train, y_train)
    valid_pool = xgb.DMatrix(x_test, y_test)
    
    # Train model
    model = xgb.XGBClassifier(**param)
    model.fit(x_train, y_train,
              eval_set=[(x_test, y_test)],
              verbose=False)

    # Predict
    preds = model.predict(x_test)
    acc = accuracy_score(y_test, preds)

    return 1.0 - acc  # Optuna minimizes

In [12]:
study = optuna.create_study(direction='minimize')  # Because we return 1 - accuracy
study.optimize(objective, n_trials=50)  # Try 50 combinations (or more!)

print("Best hyperparameters:")
print(study.best_params)

print("Best accuracy:")
print(1 - study.best_value)

[I 2025-05-31 20:50:54,172] A new study created in memory with name: no-name-d894acee-0616-4298-819f-5b0340042281
/Users/prathmesh/asep_project/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [20:50:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
[I 2025-05-31 20:50:54,647] Trial 0 finished with value: 0.03879310344827591 and parameters: {'learning_rate': 0.07801618083421254, 'n_estimators': 434, 'max_depth': 7, 'subsample': 0.6643559720679422, 'colsample_bytree': 0.8302036318968138, 'reg_alpha': 0.02186851181928189, 'reg_lambda': 0.0010589361714544233}. Best is trial 0 with value: 0.03879310344827591.
/Users/prathmesh/asep_project/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [20:50:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, 

Best hyperparameters:
{'learning_rate': 0.17763404020550788, 'n_estimators': 490, 'max_depth': 6, 'subsample': 0.6969951035211401, 'colsample_bytree': 0.7920213967702483, 'reg_alpha': 0.034912035085883496, 'reg_lambda': 0.0336597173916792}
Best accuracy:
0.978448275862069


In [13]:
# Get the data
best_model = xgb.XGBClassifier(**study.best_params)
best_model.fit(x_train, y_train,
          eval_set=[(x_test, y_test)],
          verbose=False)

# Predict
preds = best_model.predict(x_test)

# Compute final scores
acc = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds, average='binary')
recall = recall_score(y_test, preds, average='binary')
f1 = f1_score(y_test, preds, average='binary')

print("\nFinal evaluation on test set:")
print(f"Accuracy: {acc * 100:.2f}")
print(f"Precision: {precision * 100:.2f}")
print(f"Recall: {recall * 100:.2f}")
print(f"F1 Score: {f1 * 100:.2f}")



Final evaluation on test set:
Accuracy: 96.12
Precision: 94.44
Recall: 89.47
F1 Score: 91.89
